In [3]:
#import libraries

import os
import pandas as pd
import hvplot
import matplotlib
import requests
import numpy as np
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from datetime import datetime
import json
from itertools import islice

In [4]:
# load_dotenv()

In [5]:
# Set Alpaca API key and secret
alpaca_api_key =("PK938QKNYSE3UJNEDHTE")
alpaca_secret_key = ("2bsI6IeXJiK5cagd06A6yDxcfukSsbd5dF0f068x")

In [6]:
# Instantiate REST API

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    base_url = 'https://app.alpaca.markets/paper/dashboard/overview',
    api_version = "v2"
)

In [7]:
# Get user input for the stock symbol
# stock_symbol = input("Enter the stock symbol you want to analyze (e.g., AAPL): ")
stock_symbol = "META"

In [8]:
# Establish what time frame increment we will be looking at
time_frame = "1Day"

In [9]:
# Get user input for the start date in datetime format
# start_date_str = input("Enter the start date (YYYY-MM-DD): ")
# start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
start_date = pd.Timestamp("2020-10-10", tz="America/New_York").isoformat()

In [10]:
# Get user input for the end date in datetime format
# end_date_str = input("Enter the end date (YYYY-MM-DD): ")
# end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
end_date = pd.Timestamp("2020-12-12", tz="America/New_York").isoformat()

In [11]:
# trading_days = end_date - start_date


# print(f"The timeframe between {start_date_str} and {end_date_str} is {trading_days.days} days.")

In [12]:
# Calls the stock data from the period of time desired in the desired time increment
stock_data = api.get_bars(
    stock_symbol, 
    time_frame, 
    start = start_date, 
    end = end_date
).df

In [13]:
# Displays the information pulled
stock_data.info()
display(stock_data.head())
display(stock_data.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 44 entries, 2020-10-12 04:00:00+00:00 to 2020-12-11 05:00:00+00:00
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   close        44 non-null     float64
 1   high         44 non-null     float64
 2   low          44 non-null     float64
 3   trade_count  44 non-null     int64  
 4   open         44 non-null     float64
 5   volume       44 non-null     int64  
 6   vwap         44 non-null     float64
dtypes: float64(5), int64(2)
memory usage: 2.8 KB


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2020-10-12 04:00:00+00:00,275.75,280.18,267.87,292730,270.200,32999660,275.227035
2020-10-13 04:00:00+00:00,276.14,279.10,273.39,181923,277.580,19505211,276.201124
2020-10-14 04:00:00+00:00,271.82,278.75,271.50,151136,277.620,16590158,273.981850
2020-10-15 04:00:00+00:00,266.72,269.04,263.67,182305,267.600,16330684,266.033730
2020-10-16 04:00:00+00:00,265.93,271.37,265.30,157636,267.375,18430693,267.458035


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2020-12-07 05:00:00+00:00,285.58,288.4900,278.20,153281,279.190,14005842,285.751969
2020-12-08 05:00:00+00:00,283.40,286.4272,281.55,136454,286.010,11974657,283.705184
2020-12-09 05:00:00+00:00,277.92,287.6300,271.75,312698,283.660,26765687,278.642389
2020-12-10 05:00:00+00:00,277.12,278.7300,271.86,209825,275.535,21534833,276.018129
2020-12-11 05:00:00+00:00,273.55,276.4800,270.25,158853,274.530,15609459,273.330511


In [14]:
# Add new data columns to DF for body size and candle size
# body size is absolute value of open - close
# candle size is the absolute value of low - high
# clean up datafram by removing vwap, volume, trade count columns

In [15]:
# Drop columns which aren't needed(trade_count, volume, vwap) and display the results
stock_data = stock_data.drop(columns=['trade_count','volume','vwap'])
display(stock_data.head())

,close,high,low,open
timestamp,,,,
2020-10-12 04:00:00+00:00,275.75,280.18,267.87,270.200
2020-10-13 04:00:00+00:00,276.14,279.10,273.39,277.580
2020-10-14 04:00:00+00:00,271.82,278.75,271.50,277.620
2020-10-15 04:00:00+00:00,266.72,269.04,263.67,267.600
2020-10-16 04:00:00+00:00,265.93,271.37,265.30,267.375


In [16]:
stock_data.head(2)

,close,high,low,open
timestamp,,,,
2020-10-12 04:00:00+00:00,275.75,280.18,267.87,270.20
2020-10-13 04:00:00+00:00,276.14,279.10,273.39,277.58


In [17]:
# Add new data columns to DF for body size and candle size
# body size is absolute value of open - close
# candle size is the absolute value of low - high
# clean up datafram by removing vwap, volume, trade count columns

stock_data['body_size'] = stock_data['close'] - stock_data['open']
stock_data['candle_size'] = stock_data['high'] - stock_data['low']

In [18]:
display(stock_data.head())
display(stock_data.tail())

,close,high,low,open,body_size,candle_size
timestamp,,,,,,
2020-10-12 04:00:00+00:00,275.75,280.18,267.87,270.200,5.550,12.31
2020-10-13 04:00:00+00:00,276.14,279.10,273.39,277.580,-1.440,5.71
2020-10-14 04:00:00+00:00,271.82,278.75,271.50,277.620,-5.800,7.25
2020-10-15 04:00:00+00:00,266.72,269.04,263.67,267.600,-0.880,5.37
2020-10-16 04:00:00+00:00,265.93,271.37,265.30,267.375,-1.445,6.07


,close,high,low,open,body_size,candle_size
timestamp,,,,,,
2020-12-07 05:00:00+00:00,285.58,288.4900,278.20,279.190,6.390,10.2900
2020-12-08 05:00:00+00:00,283.40,286.4272,281.55,286.010,-2.610,4.8772
2020-12-09 05:00:00+00:00,277.92,287.6300,271.75,283.660,-5.740,15.8800
2020-12-10 05:00:00+00:00,277.12,278.7300,271.86,275.535,1.585,6.8700
2020-12-11 05:00:00+00:00,273.55,276.4800,270.25,274.530,-0.980,6.2300


In [19]:
# initialize variables to be used in calculations
max_body_size = stock_data["body_size"].max()
min_candle_size = stock_data["candle_size"].min()
current_count = 0
threshold = 2
stock_data["card_count"] = current_count

if stock_data.iloc[0][4] > 0:
    previous_candle = 1
elif stock_data.iloc[0][4] == 0:
    previous_candle = 0
else:
    previous_candle = -1

number_of_rows = len(stock_data)
row_counter = 0

print(max_body_size)
print(min_candle_size)
print(previous_candle)
print(stock_data.iloc[0][4])
print(number_of_rows)
stock_data.head()

8.430000000000007
3.589999999999975
1
5.550000000000011
44


C:\Users\mrjwi\AppData\Local\Temp\ipykernel_49004\1615223215.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if stock_data.iloc[0][4] > 0:
C:\Users\mrjwi\AppData\Local\Temp\ipykernel_49004\1615223215.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(stock_data.iloc[0][4])


,close,high,low,open,body_size,candle_size,card_count
timestamp,,,,,,,
2020-10-12 04:00:00+00:00,275.75,280.18,267.87,270.200,5.550,12.31,0
2020-10-13 04:00:00+00:00,276.14,279.10,273.39,277.580,-1.440,5.71,0
2020-10-14 04:00:00+00:00,271.82,278.75,271.50,277.620,-5.800,7.25,0
2020-10-15 04:00:00+00:00,266.72,269.04,263.67,267.600,-0.880,5.37,0
2020-10-16 04:00:00+00:00,265.93,271.37,265.30,267.375,-1.445,6.07,0


In [25]:
# begin for loop to start calculating each row's data to assign a +1 or -1
# to our count and add adjust our current/previous counts accordingly
#for index, rows in stock_data.iterrows():
current_count = 0
previous_close = stock_data.iloc[0][0]
previous_high = stock_data.iloc[0][1]
previous_low = stock_data.iloc[0][2]
previous_open = stock_data.iloc[0][3]
previous_body_size = stock_data.iloc[0][4]
previous_candle_size = stock_data.iloc[0][5]

#for index, row in stock_data.iterrows():
for index, row in islice(stock_data.iterrows(), 1, None):
# assign and document the current card count for the current day    
    
    if index == (number_of_rows):
        break
#assign previous day numbers to work with
    
# assign current day numbers to work with
    current_close = row[0]
    current_high = row[1]
    current_low = row[2]
    current_open = row[3]
    current_body_size = row[4]
    current_candle_size = row[5]
    
    #if current_body_size > 0:
    #    current_count = current_count + 1
    #    print(current_body_size)
    #if current_body_size < 0:
        
    #    current_count = current_count - 1
    #    print(current_body_size)

    # bullish hammer
    if (current_body_size > 0) and (current_body_size*2) < (current_candle_size):
        current_count = current_count + 1
    # bearish hammer
    elif (current_body_size < 0) and ((np.absolute(current_body_size)*2) < (current_candle_size)):
        current_count = current_count - 1
    # bullish engulfing candle
    elif (current_body_size > 0) and (previous_body_size < 1) and (current_body_size > previous_body_size):
        current_count = current_count + 1
    # bearish engulfing candle
    elif (current_body_size < 0) and (previous_body_size > 1) and (np.absolute(current_body_size) > (previous_body_size)):
        current_count = current_count - 1
    # bullish doji candle
    # elif (previous_count < 0) and (np.absolute(current_body_size) < (current_close * max_body_size)):
    #     current_count = current_count + 1


    if current_count > threshold:
        current_count = threshold
    elif current_count < -(threshold):
        current_count = -(threshold)
#



previous_close = row[0]
previous_high = row[1]
previous_low = row[2]
previous_open = row[3]
previous_body_size = row[4]
previous_candle_size = row[5]

stock_data.at[index,"card_count"] = current_count







# Create an empty DataFrame to track the trading strategy
strategy_df = pd.DataFrame(columns=["Date", "Initial_Investment", "Amount", "Number_of_Shares", "Investment_Bought", "Profits"])

strategy_df = pd.DataFrame(strategy_df)
strategy_df.head()
# Initialize 
# the initial investment
initial_investment = 10000

# Initialize other variables
amount = initial_investment
number_of_shares = 0  # No initial position
investment_bought = 0
profits = 0

# Define a list of threshold values for buy and sell
threshold = 2  # Adjust this threshold as needed
start_date = pd.Timestamp("2020-10-10", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-12-12", tz="America/New_York").isoformat()
# Calls the stock data from the period of time desired in the desired time increment
stock_data = api.get_bars(
    stock_symbol, 
    time_frame, 
    start = start_date, 
    end = end_date
).df
# Drop columns which aren't needed(trade_count, volume, vwap) and display the results
stock_data = stock_data.drop(columns=['trade_count','volume','vwap'])
display(stock_data.head())
# Define a list of dates (assuming you have a date for each trading day)
dates = ["2020-10-10", "2020-10-12", "2020-11-03", "2020-11-04", "2020-12-12"]
thresholds = [2, -3, 0, 1, -2]  # Example threshold values for each day
# Iterate through each trading day
for i in range(len(dates)):
    date = dates[i]
    threshold_value = thresholds[i]
    # Check if you can buy or sell based on the threshold
    if threshold_value > 2 and number_of_shares == 0:
        # Buy condition: Invest all the available amount
        investment_bought = amount
        number_of_shares = amount
        amount = 0  # No cash remaining
    
    elif threshold_value < -2 and number_of_shares > 0:
        # Sell condition: Sell all the shares you have
        amount += number_of_shares
        profits += amount - investment_bought
        investment_bought = 0
        number_of_shares = 0
         # Update the DataFrame with the trading day results
  # Update the DataFrame with the trading day results
strategy_df.loc[i] = {
    "Date": date,
    "Initial_Investment": initial_investment,
    "Amount": amount,
    "Number_of_Shares": number_of_shares,
    "Investment_Bought": investment_bought,
    "Profits": profits
}


# Display the trading strategy DataFrame
print(strategy_df)


    

# previous_close = row[0]
# previous_high = row[1]
# previous_low = row[2]
# previous_open = row[3]
# previous_body_size = row[4]
# previous_candle_size = row[5]

# stock_data.at[index,"card_count"] = current_count

display(stock_data.head(20))
display(stock_data.tail(10))
  

C:\Users\mrjwi\AppData\Local\Temp\ipykernel_49004\1713496406.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  previous_close = stock_data.iloc[0][0]
C:\Users\mrjwi\AppData\Local\Temp\ipykernel_49004\1713496406.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  previous_high = stock_data.iloc[0][1]
C:\Users\mrjwi\AppData\Local\Temp\ipykernel_49004\1713496406.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  previous_low = stock

,close,high,low,open
timestamp,,,,
2020-10-12 04:00:00+00:00,275.75,280.18,267.87,270.200
2020-10-13 04:00:00+00:00,276.14,279.10,273.39,277.580
2020-10-14 04:00:00+00:00,271.82,278.75,271.50,277.620
2020-10-15 04:00:00+00:00,266.72,269.04,263.67,267.600
2020-10-16 04:00:00+00:00,265.93,271.37,265.30,267.375


         Date  Initial_Investment  Amount  Number_of_Shares  \
4  2020-12-12               10000   10000                 0   

   Investment_Bought  Profits  
4                  0        0  


,close,high,low,open
timestamp,,,,
2020-10-12 04:00:00+00:00,275.75,280.18,267.87,270.2000
2020-10-13 04:00:00+00:00,276.14,279.10,273.39,277.5800
2020-10-14 04:00:00+00:00,271.82,278.75,271.50,277.6200
2020-10-15 04:00:00+00:00,266.72,269.04,263.67,267.6000
2020-10-16 04:00:00+00:00,265.93,271.37,265.30,267.3750
2020-10-19 04:00:00+00:00,261.40,268.55,259.88,265.5300
2020-10-20 04:00:00+00:00,267.56,269.70,262.88,263.0600
2020-10-21 04:00:00+00:00,278.73,283.05,276.37,279.5600
2020-10-22 04:00:00+00:00,278.12,282.45,275.04,279.8700


,close,high,low,open
timestamp,,,,
2020-11-30 05:00:00+00:00,276.97,277.7000,271.010,276.130
2020-12-01 05:00:00+00:00,286.55,289.3000,278.960,279.160
2020-12-02 05:00:00+00:00,287.52,291.7800,280.825,285.360
2020-12-03 05:00:00+00:00,281.85,286.6500,281.070,286.250
2020-12-04 05:00:00+00:00,279.70,283.4600,279.300,280.300
2020-12-07 05:00:00+00:00,285.58,288.4900,278.200,279.190
2020-12-08 05:00:00+00:00,283.40,286.4272,281.550,286.010
2020-12-09 05:00:00+00:00,277.92,287.6300,271.750,283.660
2020-12-10 05:00:00+00:00,277.12,278.7300,271.860,275.535


In [24]:
stock_data.reset_index(inplace = True)
stock_data.head()

,timestamp,close,high,low,open,card_count
0,2020-10-12 04:00:00+00:00,275.75,280.18,267.87,270.200,NaN
1,2020-10-13 04:00:00+00:00,276.14,279.10,273.39,277.580,NaN
2,2020-10-14 04:00:00+00:00,271.82,278.75,271.50,277.620,NaN
3,2020-10-15 04:00:00+00:00,266.72,269.04,263.67,267.600,NaN
4,2020-10-16 04:00:00+00:00,265.93,271.37,265.30,267.375,NaN
